# Lecture 7 - 22.11.2021
# Exercise : The Pol-InSAR Coherence Region 

* DLR's F-SAR acquisition over Traunstein forest (Germany)
* Path: '/projects/data/04-polinsar/'
* SLCs: 
    * Acquisition 1 : slc_15tmpsar0302_L { hh, hv, vv, vh } _t01.rat
    * Acquisition 2 : slc_15tmpsar0302_15tmpsar0303_L { hh, hv, vv, vh } _t01.rat
* Flat-earth: pha_flat_15tmpsar0302_15tmpsar0303_Lhh_t01.rat
* Vertical wavenumber (kz) : kz_2d_demc_15tmpsar0302_15tmpsar0303_t01.rat
* Lidar: Lida_r1503.rat

Objective:
- Calculate the dual-pol Pol-InSAR coherence region parameters, find the 2 extremes, and visualize them together with the phase difference. In addition, visualize the Pol-InSAR coherence region in one or more points.

Tips:
- work on the azimuth - range block [21500 - 4000, 21500 + 6000] - [2300, 4500] ;
- all the needed functions and a few pieces of script have been already implemented.

In [ ]:
# import useful libraries, functions, and modules

import sys
sys.path.append('/projects/src/')

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import filters
from ste_io import *

%matplotlib widget




In [ ]:
def calculate_covariance(im1, im2, looksr, looksa) : 
    
    # ... apply definition
    corr = filters.uniform_filter(np.real(im1*np.conj(im2)), [looksa,looksr]) + 1j* \
                filters.uniform_filter(np.imag(im1*np.conj(im2)), [looksa,looksr])
    
    # ... and back to main
    return corr

In [ ]:
def make_pauli(slchh, slchv, slcvv, looksr, looksa) :
    
    # 1. Uses function calculate_covariance
    # 2. Convention (rows x columns): inputs are (az x rg), outputs are (rg x az) 
    #                                 for better plotting as #rg <#az
    
    # Calculate T11, T22 and T33
    T11 = calculate_covariance(slchh + slcvv, slchh + slcvv, looksr, looksa)
    T22 = calculate_covariance(slchh - slcvv, slchh - slcvv, looksr, looksa)
    T33 = calculate_covariance(2*slchv, 2*slchv, looksr, looksa)
    
    # make the pauli rgb (+ tranpose, clipping, normalization)
    dimaz = T11.shape[0]
    dimrg = T11.shape[1]
    rgb_pauli = np.zeros((dimrg, dimaz, 3), 'float32')
    rgb_pauli[:, :, 0] = np.transpose( np.clip(np.sqrt(np.abs(T22)), 0, 2.5*np.mean(np.sqrt(np.abs(T22)))) )    # red
    rgb_pauli[:, :, 1] = np.transpose( np.clip(np.sqrt(np.abs(T33)), 0, 2.5*np.mean(np.sqrt(np.abs(T33)))) )    # green
    rgb_pauli[:, :, 2] = np.transpose( np.clip(np.sqrt(np.abs(T11)), 0, 2.5*np.mean(np.sqrt(np.abs(T11)))) )    # blue
    rgb_pauli[:, :, 0] = rgb_pauli[:, :, 0] / np.max(rgb_pauli[:, :, 0])     # red
    rgb_pauli[:, :, 1] = rgb_pauli[:, :, 1] / np.max(rgb_pauli[:, :, 1])     # green
    rgb_pauli[:, :, 2] = rgb_pauli[:, :, 2] / np.max(rgb_pauli[:, :, 2])     # blue
    
    # ... and back to main
    return rgb_pauli

In [ ]:
def matmul_2(a11, a12, a21, a22, b11, b12, b21, b22) :
    
    # Implements the multiplication A.B of two 2 x 2 matrices A and B.
    # Inputs are the elements of matrices (row-wise). 
    # Outputs are the elements of the matrix product (row-wise).
    
    c11 = a11*b11 + a12*b21
    c12 = a11*b12 + a12*b22
    c21 = a21*b11 + a22*b21 
    c22 = a21*b12 + a22*b22
    
    # ... and back to main
    return c11, c12, c21, c22

In [ ]:
def eigenvalvect_2(a11, a12, a21, a22) :
    
    # Calculates eigenvalues / eigenvectors of a generic complex-valued 2 x 2 matrix.
    # Inputs are the matrix elements (row-wise) - NumPy arrays
    # Outputs are the eigenvalues (l1, l2), and the eigenvector (E1, E2) elements - NumPy arrays
    #                                       E1 = [e11, e21] , U2 = [e12, e22]
    
    # eigenvalues 
    l1 = 0.5 * ((a11+a22) + np.sqrt( (a11-a22)**2 + 4*a12*a21 )) 
    l2 = 0.5 * ((a11+a22) - np.sqrt( (a11-a22)**2 + 4*a12*a21 ))
    
    # eigenvector 1
    e11 = -a12
    e21 = a11 - l1
    # normalize eigenvector 1
    norm_ = np.sqrt( np.abs(e11)**2 + np.abs(e21)**2 )
    e11 = e11 / norm_
    e21 = e21 / norm_
    # eigenvector 2
    e12 = a22 - l2
    e22 = -a21
    # normalize eigenvector 2
    norm_ = np.sqrt( np.abs(e12)**2 + np.abs(e22)**2 )
    e12 = e12 / norm_
    e22 = e22 / norm_
    
    del norm_
    
    # ... and back to main
    return l1, l2, e11, e21, e12, e22

In [ ]:
def off_diagonal_2(a11, a12, a21, a22, e11, e21, e12, e22) :

    # Calculates off-diagonal form a 2 x 2 matrix A given its eigenvalues in E
    # Inputs are the matrix elements (row-wise) - NumPy arrays
    #        and the eigenvector (E1, E2) elements - NumPy arrays
    #                                       E1 = [e11, e21] , U2 = [e12, e22]
    # Outputs are the off-diagonal elements: o12, o21
    
    # rotation angle theta
    theta = np.angle( np.conj(e11)*e12 + np.conj(e21)*e22  )
    
    # first column of the off-diagonalizing matrix (normalized)
    u11 = np.exp(1j*theta)*e11 + e12
    u21 = np.exp(1j*theta)*e21 + e22
    norm_ = np.sqrt( np.abs(u11)**2 + np.abs(u21)**2 )
    u11 = u11 / norm_
    u21 = u21 / norm_
    
    # second column of the off-diagonalizing matrix (normalized)
    u12 = np.exp(1j*theta)*e11 - e12
    u22 = np.exp(1j*theta)*e21 - e22
    norm_ = np.sqrt( np.abs(u12)**2 + np.abs(u22)**2 )
    u12 = u12 / norm_
    u22 = u22 / norm_
   
    del norm_
    
    # calculate off-diagonal elements
    o12 = a11 * ( np.conj(u11)*u12 - np.conj(u21)*u22 ) +\
          a12 * np.conj(u11) * u22 +\
          a21 * np.conj(u21) * u12
    o21 = a11 * ( np.conj(u12)*u11 - np.conj(u22)*u21 ) +\
          a12 * np.conj(u12) * u21 +\
          a21 * np.conj(u22) * u11
    
    # ... and back to main
    return o12, o21

In [ ]:
def sqrt_inverse(a11, a12, a22) :

    # Given a 2 x 2 Hermitian matrix A, calculates A**(-1/2) using eigendecomposition
    # Inputs are the upper-diagonal matrix elements (row-wise) - NumPy arrays.
    # Outputs are the elements of the sqrt inverse matrix.
    # Uses function matmul_2.
    
    # Calculate eigenvalues (l1, l2)
    sigma = 0.5 * (a11 + a22)
    D = a11*a22 - a12*np.conj(a12)
    delta = np.sqrt(sigma**2 - D)
    l1 = sigma + delta
    l2 = sigma - delta
    
    # Calculate eigenvector matrix elements (eigenvectors are the columns)
    norm_ = np.sqrt( a12*np.conj(a12) + ((a11-a22)/2 - delta)**2 )
    v11 = -a12 / norm_
    v21 = ((a11-a22)/2 - delta) / norm_
    v12 = ((a11-a22)/2 - delta) / norm_
    v22 = np.conj(a12) / norm_
    
    del norm_, delta, D, sigma
    
    # calculate sqrt inverse matrix elements
    zz = np.zeros(l1.shape, 'float32')
    aux11, aux12, aux21, aux22 = matmul_2(v11, v12, v21, v22, 1/np.sqrt(l1), zz, zz, 1/np.sqrt(l2))
    del l1, l2
    i11, i12, i21, i22 = matmul_2(aux11, aux12, aux21, aux22, np.conj(v11), np.conj(v21), np.conj(v12), np.conj(v22))
    del aux11, aux12, aux21, aux22
    
    # ... and back to main
    return i11, i12, i21, i22  

**Input parameters**

In [ ]:
# --- Inputs

# path 2 acquisitions
path = '/projects/data/04-polinsar/'

# Input pixel spacing, in meters
spacrg = 0.59941552
spacaz = 0.19507939

# Output range resolution, in meters
resrg = 5.
resaz = 5.

# Image block for processing
minrg = 2300
maxrg = 4500
minaz = 21500 - 4000
maxaz = 21500 + 6000

In [ ]:
# --- Calculate number of looks

looksr = int( resrg / spacrg )
if looksr % 2 == 0 : looksr = looksr +1
looksa = int( resaz / spacaz )
if looksa % 2 == 0 : looksa = looksa +1


**Step 1: Open images, and visualize a Pauli**

**Step 2 : Compensate flat-earth**

In [ ]:
# --- Open flat-earth
fe = rrat(path + 'pha_flat_15tmpsar0302_15tmpsar0303_Lhh_t01.rat', block = [minaz, maxaz, minrg, maxrg])

# --- compensate
slchh2 = slchh2 * np.exp(1j*fe)
slchv2 = slchv2 * np.exp(1j*fe)
slcvv2 = slcvv2 * np.exp(1j*fe)

**Step 3 : Calculate the elements of T1, T2, and Omega matrices**

**Step 4 : Pre-whitening and centering**

In [ ]:
# --- Pre-whitening

# calculate T
T11 = 0.5 * ( T1_11 + T2_11 )
T12 = 0.5 * ( T1_12 + T2_12 )
T22 = 0.5 * ( T1_22 + T2_22 )

# free memory
del T1_11, T1_12, T1_22, T2_11, T2_12, T2_22

# calculate the elements of T**(-1/2)
iT11, iT12, iT21, iT22 = sqrt_inverse(T11, T12, T22)

# free memory
del T11, T12, T22

# whiten ... two 2 x 2 multiplications ==> Matrix P
aux11, aux12, aux21, aux22 = matmul_2(iT11, iT12, iT21, iT22, Om_11, Om_12, Om_21, Om_22)
del Om_11, Om_12, Om_21, Om_22
P11, P12, P21, P22 = matmul_2(aux11, aux12, aux21, aux22, iT11, iT12, iT21, iT22)
del aux11, aux12, aux21, aux22
del iT11, iT12, iT21, iT22

# --- Centering... translate P by 0.5*trace(P) * Identity ==> Matrix O

center = 0.5 * (P11 + P22)  # it is the center of the coherence region !!
O11 = P11 - center
O12 = P12
O21 = P21
O22 = P22 - center

# free memory
del P11, P12, P21, P22

**Step 5 : Calculate geometry of the coherence region**

**Step 6 : Calculate extreme coherences**

In [ ]:
# --- angular extremes  


# --- radial extremes




**Step 7 : Plots !**

In [ ]:
# --- open lidar height



In [ ]:
# --- Plot phase differences... converted to height


In [ ]:
# --- Plot height + coherences
